# Direct Preference Alignment (DPO) with PyTorch FSDP on Amazon SageMaker AI using ModelTrainer

In this notebook, we run a DPO workload on Amazon SageMaker AI, using Python scripts and SageMaker ModelTrainer for executing a training job.

## Prerequisites

In [ ]:
%pip install -r ./scripts/requirements.txt --upgrade

***

## Setup Configuration file path

If you have created a Managed MLflow server, copy the `ARN` code here and assign a name to the experiment

In [ ]:
import os

model_id = "Qwen/Qwen3-0.6B"

os.environ["model_id"] = model_id
os.environ["mlflow_uri"] = ""
os.environ["mlflow_experiment_name"] = "qwen3-06b-dpo"

***

## Visualize and upload the dataset

In this example, we are going to load [nvidia/When2Call](https://huggingface.co/datasets/nvidia/When2Call) dataset, an open-source dataset and model suite focused on enabling and improving function calling capabilities for large language models (LLMs).

In [ ]:
import sagemaker

In [ ]:
sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix

In [ ]:
from datasets import load_dataset

dataset = load_dataset("nvidia/When2Call", "train_pref", split="train[:4000]")

dataset

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset)

df.head()

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1, random_state=42)

print("Number of train elements: ", len(train))
print("Number of test elements: ", len(test))

Utility functions to format messages

In [ ]:
import json
import re
from transformers import AutoTokenizer
import uuid

tokenizer = AutoTokenizer.from_pretrained(model_id)


def extract_system_message(text):
    start_marker = "<|im_start|>system"
    end_marker = "<|im_end|>"

    start_index = text.find(start_marker) + len(start_marker)
    end_index = text.find(end_marker, start_index)

    if start_index >= 0 and end_index >= 0:
        return text[start_index:end_index].strip()
    else:
        return None


def extract_toolcall_content(text):
    """Extract content between <TOOLCALL> tags if present."""
    if isinstance(text, dict):
        if text.get("content"):
            text = text.get("content")

    found_tools = False
    if isinstance(text, str) and "<TOOLCALL>" in text and "</TOOLCALL>" in text:
        pattern = r"<TOOLCALL>(.*?)</TOOLCALL>"
        match = re.search(pattern, text, re.DOTALL)
        if match:
            found_tools = True
            tool_calls = []

            if isinstance(match.group(1), str):
                tools = json.loads(match.group(1))

            for tool_call in tools:
                arguments = (
                    json.loads(tool_call["arguments"])
                    if isinstance(tool_call["arguments"], str)
                    else tool_call["arguments"]
                )

                tool_call_json = {
                    "id": str(uuid.uuid4())[:9],
                    "name": tool_call["name"],
                    "parameters": arguments,
                }

                tool_calls.append({"type": "function", "function": tool_call_json})

            return found_tools, tool_calls
    return found_tools, text


def transform_tool_format(tool):
    """Transform tool from old format to Nova format."""
    if isinstance(tool, str):
        tool = json.loads(tool)

    return {
        "type": "function",
        "function": {
            "name": tool["name"],
            "description": tool["description"],
            "parameters": tool["parameters"],
        },
    }

Prepare the chat completion arrays with chosen and rejected answers

In [ ]:
def prepare_dataset(sample):
    try:
        # Parse tools only once if needed
        if isinstance(sample["tools"], str):
            tools = json.loads(sample["tools"]) if sample.get("tools") else []
        else:
            tools = sample["tools"]

        transformed_tools = [transform_tool_format(tool) for tool in tools]

        # Define message transformations based on role
        messages = []
        system_text = "You are a helpful assistant."

        messages.append({"role": "system", "content": system_text})

        for message in sample.get("messages", []):
            if message["role"] == "user":
                messages.append({"role": "user", "content": message["content"]})
            elif message["role"] == "tool":
                print(message)
            elif message["role"] == "assistant":
                found_tools, content = extract_toolcall_content(message["content"])
                if found_tools:
                    messages.append(
                        {"role": "tool", "content": "", "tool_calls": content}
                    )
                else:
                    messages.append({"role": "assistant", "content": content})

        prompt = tokenizer.apply_chat_template(
            messages,
            tools=transformed_tools,
            tokenize=False,
        )

        found_tools, chosen_text = extract_toolcall_content(sample["chosen_response"])
        final_chosen_text = "<|im_start|>assistant"
        if found_tools:
            for tool in chosen_text:
                tool_formatted = {
                    "name": tool["function"]["name"],
                    "parameters": tool["function"]["parameters"],
                }

                final_chosen_text = final_chosen_text + "<tool_call>\n" + json.dumps(tool_formatted) + "\n</tool_call>\n"
        else:
            final_chosen_text = final_chosen_text + "\n" + chosen_text

        final_chosen_text = final_chosen_text + "<|im_end|>"

        found_tools, rejected_text = extract_toolcall_content(sample["rejected_response"])
        final_rejected_text = "<|im_start|>assistant"
        if found_tools:
            for tool in rejected_text:
                tool_formatted = {
                    "name": tool["function"]["name"],
                    "parameters": tool["function"]["parameters"],
                }

                final_rejected_text = final_rejected_text + "<tool_call>\n" + json.dumps(tool_formatted) + "\n</tool_call>\n"
        else:
            final_rejected_text = final_rejected_text + "\n" + rejected_text

        final_rejected_text = final_rejected_text + "<|im_end|>"

        return {
            "prompt": prompt,
            "chosen": final_chosen_text,
            "rejected": final_rejected_text,
        }
    except Exception as e:
        print(f"Error: {e}")

        raise e

In [ ]:
from datasets import Dataset, DatasetDict
from random import randint

train_dataset = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

dataset = DatasetDict({"train": train_dataset, "test": test_dataset})

train_dataset = dataset["train"].map(
    prepare_dataset, remove_columns=list(train_dataset.features)
)

print(train_dataset[randint(0, len(train_dataset))])

test_dataset = dataset["test"].map(
    prepare_dataset, remove_columns=test_dataset.features
)

### Upload to Amazon S3

In [ ]:
import boto3
import shutil
import sagemaker

In [ ]:
sagemaker_session = sagemaker.Session()
s3_client = boto3.client('s3')

bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix

In [ ]:
# save train_dataset to s3 using our SageMaker session
if default_prefix:
    input_path = f"{default_prefix}/datasets/llm-fine-tuning-modeltrainer-dpo"
else:
    input_path = f"datasets/llm-fine-tuning-modeltrainer-dpo"

train_dataset_s3_path = f"s3://{bucket_name}/{input_path}/train/dataset.json"
test_dataset_s3_path = f"s3://{bucket_name}/{input_path}/test/dataset.json"

In [ ]:
# Save datasets to s3
# We will fine tune only with 20 records due to limited compute resource for the workshop
train_dataset.to_json("./data/train/dataset.json", orient="records")
test_dataset.to_json("./data/test/dataset.json", orient="records")

s3_client.upload_file("./data/train/dataset.json", bucket_name, f"{input_path}/train/dataset.json")
s3_client.upload_file("./data/test/dataset.json", bucket_name, f"{input_path}/test/dataset.json")

shutil.rmtree("./data")

print(f"Training data uploaded to:")
print(train_dataset_s3_path)
print(test_dataset_s3_path)

***

## Model fine-tuning

We are now ready to fine-tune our model. We will use the [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) from transfomers to fine-tune our model. We prepared a script [train.py](./scripts/train.py) which will loads the dataset from disk, prepare the model, tokenizer and start the training.

For configuration we use `TrlParser`, that allows us to provide hyperparameters in a `yaml` file. This yaml will be uploaded and provided to Amazon SageMaker similar to our datasets. We are saving the config file as `args.yaml` and upload it to S3.

In [ ]:
%%bash

cat > ./args.yaml <<EOF
model_id: "${model_id}"       # Hugging Face model id
mlflow_uri: "${mlflow_uri}"
mlflow_experiment_name: "${mlflow_experiment_name}"
# sagemaker specific parameters
output_dir: "/opt/ml/model"                       # path to where SageMaker will upload the model 
checkpoint_dir: "/opt/ml/checkpoints/"
train_dataset_path: "/opt/ml/input/data/train/"   # path to where FSx saves train dataset
test_dataset_path: "/opt/ml/input/data/test/"     # path to where FSx saves test dataset
save_steps: 100                                   # Save checkpoint every this many steps
# training parameters
lora_r: 8
lora_alpha: 16
lora_dropout: 0.1                 
learning_rate: 2e-4                    # learning rate scheduler
num_train_epochs: 2                    # number of training epochs
per_device_train_batch_size: 2         # batch size per device during training
per_device_eval_batch_size: 1          # batch size for evaluation
gradient_accumulation_steps: 2         # number of steps before performing a backward/update pass
gradient_checkpointing: true           # use gradient checkpointing
bf16: true                             # use bfloat16 precision
tf32: false                            # use tf32 precision
fsdp: "full_shard auto_wrap offload"
fsdp_config: 
    backward_prefetch: "backward_pre"
    cpu_ram_efficient_loading: true
    offload_params: true
    forward_prefetch: false
    use_orig_params: true
merge_weights: true                    # merge weights in the base model
EOF

Lets upload the config file to S3.

In [ ]:
from sagemaker.s3 import S3Uploader

if default_prefix:
    input_path = f"s3://{bucket_name}/{default_prefix}/datasets/llm-fine-tuning-modeltrainer-dpo"
else:
    input_path = f"s3://{bucket_name}/datasets/llm-fine-tuning-modeltrainer-dpo"

# upload the model yaml file to s3
model_yaml = "args.yaml"
train_config_s3_path = S3Uploader.upload(local_path=model_yaml, desired_s3_uri=f"{input_path}/config")

print(f"Training config uploaded to:")
print(train_config_s3_path)

## Fine-tune model

Below estimtor will train the model with QLoRA, merge the adapter in the base model and save in S3

#### Get PyTorch image_uri

We are going to use the native PyTorch container image, pre-built for Amazon SageMaker

In [ ]:
import sagemaker
from sagemaker.config import load_sagemaker_config

In [ ]:
sagemaker_session = sagemaker.Session()

bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix
configs = load_sagemaker_config()

In [ ]:
instance_type = "ml.g5.12xlarge" # Override the instance type if you want to get a different container version
instance_count = 1

instance_type

In [ ]:
image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=sagemaker_session.boto_session.region_name,
    version="2.6.0",
    instance_type=instance_type,
    image_scope="training"
)

image_uri

In [ ]:
from sagemaker.modules.configs import (
    CheckpointConfig,
    Compute,
    OutputDataConfig,
    SourceCode,
    StoppingCondition,
)
from sagemaker.modules.distributed import Torchrun
from sagemaker.modules.train import ModelTrainer

# Define the script to be run
source_code = SourceCode(
    source_dir="./scripts",
    requirements="requirements.txt",
    entry_script="train.py",
)

# Define the compute
compute_configs = Compute(
    instance_type=instance_type,
    instance_count=instance_count,
    keep_alive_period_in_seconds=0,
)

# define Training Job Name
job_name = f"train-{model_id.split('/')[-1].replace('.', '-')}-dpo"

# define OutputDataConfig path
if default_prefix:
    output_path = f"s3://{bucket_name}/{default_prefix}/{job_name}"
else:
    output_path = f"s3://{bucket_name}/{job_name}"

# Define the ModelTrainer
model_trainer = ModelTrainer(
    training_image=image_uri,
    source_code=source_code,
    base_job_name=job_name,
    compute=compute_configs,
    distributed=Torchrun(),
    stopping_condition=StoppingCondition(max_runtime_in_seconds=7200),
    hyperparameters={
        "config": "/opt/ml/input/data/config/args.yaml"  # path to TRL config which was uploaded to s3
    },
    output_data_config=OutputDataConfig(s3_output_path=output_path),
    checkpoint_config=CheckpointConfig(
        s3_uri=output_path + "/checkpoint", local_path="/opt/ml/checkpoints"
    ),
)

In [ ]:
from sagemaker.modules.configs import InputData

# Pass the input data
train_input = InputData(
    channel_name="train",
    data_source=train_dataset_s3_path, # S3 path where training data is stored
)

test_input = InputData(
    channel_name="test",
    data_source=test_dataset_s3_path, # S3 path where training data is stored
)

config_input = InputData(
    channel_name="config",
    data_source=train_config_s3_path, # S3 path where training data is stored
)

# Check input channels configured
data = [train_input, test_input, config_input]
data

In [ ]:
# starting the train job with our uploaded datasets as input
model_trainer.train(input_data_config=data, wait=False)

***

# Model Deployment

In the following sections, we are going to deploy the fine-tuned model on an Amazon SageMaker Real-time endpoint.

## Load Fine-Tuned model

In [ ]:
import boto3
import sagemaker

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
model_id = "Qwen/Qwen3-0.6B"

bucket_name = sagemaker_session.default_bucket()
default_prefix = sagemaker_session.default_bucket_prefix
job_prefix = f"train-{model_id.split('/')[-1].replace('.', '-')}-dpo"

In [ ]:
def get_last_job_name(job_name_prefix):
    sagemaker_client = boto3.client('sagemaker')

    matching_jobs = []
    next_token = None

    while True:
        # Prepare the search parameters
        search_params = {
            'Resource': 'TrainingJob',
            'SearchExpression': {
                'Filters': [
                    {
                        'Name': 'TrainingJobName',
                        'Operator': 'Contains',
                        'Value': job_name_prefix
                    },
                    {
                        'Name': 'TrainingJobStatus',
                        'Operator': 'Equals',
                        'Value': "Completed"
                    }
                ]
            },
            'SortBy': 'CreationTime',
            'SortOrder': 'Descending',
            'MaxResults': 100
        }

        # Add NextToken if we have one
        if next_token:
            search_params['NextToken'] = next_token

        # Make the search request
        search_response = sagemaker_client.search(**search_params)

        # Filter and add matching jobs
        matching_jobs.extend([
            job['TrainingJob']['TrainingJobName'] 
            for job in search_response['Results']
            if job['TrainingJob']['TrainingJobName'].startswith(job_name_prefix)
        ])

        # Check if we have more results to fetch
        next_token = search_response.get('NextToken')
        if not next_token or matching_jobs:  # Stop if we found at least one match or no more results
            break

    if not matching_jobs:
        raise ValueError(f"No completed training jobs found starting with prefix '{job_name_prefix}'")

    return matching_jobs[0]

In [ ]:
job_name = get_last_job_name(job_prefix)

job_name

#### Inference configurations

In [ ]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker import Model

In [ ]:
instance_count = 1
instance_type = "ml.g5.xlarge"
number_of_gpu = 1
health_check_timeout = 700

In [ ]:
image_uri = sagemaker.image_uris.retrieve(
    framework="djl-lmi",
    region=sagemaker_session.boto_session.region_name,
    version="latest"
)

image_uri = image_uri.split("/")[0] + "/djl-inference:0.33.0-lmi15.0.0-cu128"

image_uri

In [ ]:
if default_prefix:
    model_data=f"s3://{bucket_name}/{default_prefix}/{job_prefix}/{job_name}/output/model.tar.gz"
else:
    model_data=f"s3://{bucket_name}/{job_prefix}/{job_name}/output/model.tar.gz"

model = Model(
    image_uri=image_uri,
    model_data=model_data,
    role=get_execution_role(),
    env={
        "HF_MODEL_ID": "/opt/ml/model",  # path to where sagemaker stores the model
        "OPTION_TRUST_REMOTE_CODE": "true",
        "OPTION_ROLLING_BATCH": "vllm",
        "OPTION_DTYPE": "bf16",
        "OPTION_QUANTIZE": "fp8",
        "OPTION_TENSOR_PARALLEL_DEGREE": "max",
        "OPTION_MAX_ROLLING_BATCH_SIZE": "32",
        "OPTION_MODEL_LOADING_TIMEOUT": "3600",
        "OPTION_ENABLE_AUTO_TOOL_CHOICE": "true",
        "OPTION_TOOL_CALL_PARSER": "hermes",
        "OPTION_ENABLE_STREAMING": "true",
    },
)

In [ ]:
endpoint_name = f"{model_id.split('/')[-1].replace('.', '-')}-djl"

In [ ]:
predictor = model.deploy(
    endpoint_name=endpoint_name,
    initial_instance_count=instance_count,
    instance_type=instance_type,
    container_startup_health_check_timeout=health_check_timeout,
    model_data_download_timeout=3600
)

#### Predict

In [ ]:
import sagemaker

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
model_id = "Qwen/Qwen3-0.6B"

endpoint_name = f"{model_id.split('/')[-1].replace('.', '-')}-djl"

In [ ]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [ ]:
import json

system_prompt = f"""
You are a helpful AI assistant that can answer questions and provide information.
You must include your reasoning activities in the tags <thinking></thinking>
You must include your final answer in the tags <answer></answer>
You can use tools to help you with your tasks.
"""

tools = [
    {
        "type": "function",
        "function": {
            "name": "calculate_bmi",
            "description": "Calculate BMI given weight in kg and height in meters",
            "parameters": {
                "type": "object",
                "properties": {
                    "weight_kg": {
                        "type": "number",
                        "description": "Property weight_kg",
                    },
                    "height_m": {"type": "number", "description": "Property height_m"},
                },
                "required": ["weight_kg", "height_m"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "fetch_weather",
            "description": 'Fetch weather information\n\nArgs:\nquery: The weather query (e.g., "weather in New York")\nnum_results: Number of results to return (default: 1)\n\nReturns:\nJSON string containing weather information\n',
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Property query"},
                    "num_results": {
                        "type": "integer",
                        "description": "Property num_results",
                    },
                },
                "required": ["query"],
            },
        },
    },
]

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What is the weather in Rome, Italy?"},
]

response = predictor.predict(
    {
        "messages": messages,
        "temperature": 0.1,
        "max_tokens": 4096,
        "tools": tools
    }
)

response

#### Delete Endpoint

In [ ]:
import sagemaker

In [ ]:
sagemaker_session = sagemaker.Session()

In [ ]:
model_id = "Qwen/Qwen3-0.6B"

endpoint_name = f"{model_id.split('/')[-1].replace('.', '-')}-djl"

In [ ]:
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sagemaker_session,
    serializer=sagemaker.serializers.JSONSerializer(),
    deserializer=sagemaker.deserializers.JSONDeserializer(),
)

In [ ]:
predictor.delete_model()
predictor.delete_endpoint(delete_endpoint_config=True)